In [108]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from scipy.sparse import rand as sprand

In [52]:
movie_data = pd.read_csv('data_files/sorted_movie_data.csv')
movie_data.drop(['Unnamed: 0'], axis=1, inplace=True)
movie_data = movie_data.groupby('movie_id',as_index=False,sort=False).last()
movie_data

,movie_id,title,release_date,imdb_url,genres,user_id,rating,timestamp
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Animation|Children's|Comedy,593,3,875659150
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Action|Adventure|Thriller,889,3,880182460
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,Thriller,880,1,880175023
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,Action|Comedy|Drama,870,2,879270213
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),Crime|Drama|Thriller,880,3,880241379
...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,Drama,863,1,889289570
1678,1679,B. Monkey (1998),06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,Romance|Thriller,863,3,889289491
1679,1680,Sliding Doors (1998),01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),Drama|Romance,863,2,889289570
1680,1681,You So Crazy (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,Comedy,896,3,887160722


In [66]:
movies_SA = pd.read_csv('data_files/movie_tweets_with_SA1.csv')
movies_SA.drop(['Unnamed: 0','Title'], axis=1, inplace=True)
movies_SA.rename(columns={"Movie Id": "movie_id"}, inplace=True)
movies_SA = movies_SA.groupby('movie_id',as_index=False,sort=False).last()
movies_SA

,movie_id,Tagword,description,cleaned_tweets,Sentiment_score,Sentiment_label
0,28,#apollo13,I’m a theatre kid who loves space. Proud Democ...,I ’ m a who space Democratic Artist Painter Ar...,0.9992,Positive
1,28,#apollo13,"Poet, Storyteller... at least trying to tell one.",Poet Storyteller at least trying to tell one w...,0.9988,Positive
2,28,#apollo13,An open source bot here to support all women w...,An open source bot here to support all who cod...,0.9983,Positive
3,28,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,Our mission is to inspire you to excel in tech...,0.9991,Positive
4,28,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,Log On And Stay With Steven Da Latest Merchand...,0.9989,Positive
...,...,...,...,...,...,...
3955,66,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,Batman Forever Twitter Bot An community for of...,0.9974,Positive
3956,66,#whileyouweresleeping,Do not lead me down this road unless you’re su...,to excellence in the art form Everything about...,0.9845,Positive
3957,66,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",All I do is tweet favorite and Oh and share my...,0.9980,Positive
3958,66,#whileyouweresleeping,simple in all aspect of life,in Finance Author of paranormal suspense myste...,0.9975,Positive


In [69]:
common = movie_data.merge(movies_SA, on=["movie_id"])
Ncommon = movie_data[~movie_data.movie_id.isin(common.movie_id)]

,movie_id,title,release_date,imdb_url,genres,user_id,rating,timestamp
7,8,Babe (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Babe%20(1995),Children's|Comedy|Drama,889,3,880179757
12,13,Mighty Aphrodite (1995),30-Oct-1995,http://us.imdb.com/M/title-exact?Mighty%20Aphr...,Comedy,716,2,879793376
13,14,"Postino, Il (1994)",01-Jan-1994,"http://us.imdb.com/M/title-exact?Postino,%20Il...",Drama|Romance,150,4,878746889
14,15,Mr. Holland's Opus (1995),29-Jan-1996,http://us.imdb.com/M/title-exact?Mr.%20Holland...,Drama,901,5,877130439
15,16,French Twist (Gazon maudit) (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Gazon%20maudi...,Comedy|Romance,894,3,880993614
...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,Drama,863,1,889289570
1678,1679,B. Monkey (1998),06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,Romance|Thriller,863,3,889289491
1679,1680,Sliding Doors (1998),01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),Drama|Romance,863,2,889289570
1680,1681,You So Crazy (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,Comedy,896,3,887160722


In [73]:
cols = ['Tagword','description','cleaned_tweets', 'Sentiment_score', 'Sentiment_label']
Ncommon[cols[0]] = np.nan
Ncommon[cols[1]] = np.nan
Ncommon[cols[2]] = np.nan
Ncommon[cols[3]] = 0
Ncommon[cols[4]] = 0

<ipython-input-73-5a1d6bba1239>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ncommon[cols[0]] = np.nan
<ipython-input-73-5a1d6bba1239>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ncommon[cols[1]] = np.nan
<ipython-input-73-5a1d6bba1239>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [76]:
all_data = common.append(Ncommon, ignore_index = True)

In [82]:
n_users = all_data.user_id.unique().shape[0]
print(n_users)

489


In [83]:
n_items = all_data.movie_id.unique().shape[0]
print(n_items)

1682


In [80]:
u_genre.id.unique().shape[0]

19

In [84]:
ratings = sprand(n_users, n_items, density=0.01, format="csr")
ratings.data = np.random.randint(1, 5, size=ratings.nnz).astype(np.float64)
ratings = ratings.toarray()

In [85]:
# class MatrixFactorization(torch.nn.Module):
#     def __init__(self, n_users, n_items, n_factors=20):
#         super().__init__()
#         self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
#         self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

#     def forward(self, user, item):
#         return (self.user_factors(user) * self.item_factors(item)).sum(1)
    

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

        self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=1, kernel_size=1)  
      
        self.pool = torch.nn.MaxPool2d(2,2)
        self.conv2 = torch.nn.Conv2d(1, 16, 1)    
        self.fc1 = torch.nn.Linear(32*20, 128)    
        self.fc2 = torch.nn.Linear(128, 64)       
        self.fc3 = torch.nn.Linear(64, 1)                  # Fully connected layer,with output dimension = 1


    def forward(self, user, item):
        user = self.user_factors(user) 
        item = self.item_factors(item)

        user = [t.detach().numpy() for t in user]
        item = [t.detach().numpy() for t in item]

        input_val = torch.LongTensor([user, item])

        input_val = torch.unsqueeze(input_val.long(), dim=1) 
        input_val = torch.tensor(input_val,dtype=torch.float)
      
        x = F.relu(self.conv2(input_val))
        x = x.view( -1,32*20)
        x = F.relu(self.fc1(x))           
        x = F.relu(self.fc2(x))           
        x = F.relu(self.fc3(x))           

        return x

In [86]:
model = MatrixFactorization(n_users, n_items) #, n_factors=20)

In [87]:
loss_func = torch.nn.MSELoss()

In [88]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)  # learning rate

In [89]:
results = all_data.to_numpy()

In [112]:
rows, cols = results.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

loss=[]
predictions = []
true_value = []

step = 0
for row, col in zip(*(rows, cols)):
    # Set gradients to zero
    optimizer.zero_grad()
    true_value.append(results[row, col])
    
    # Turn data into tensors
    rating = torch.FloatTensor([results[row, col]])
    
    row = torch.LongTensor([row])
    col = torch.LongTensor([col])

    # Predict and calculate loss
    prediction_ = model(row, col)
    
    predictions.append(float(prediction_))

    loss_ = loss_func(prediction_, rating)
    loss.append(float(loss_))

    # Backpropagate
    loss_.backward()

    rmse = math.sqrt(float(loss_))
    print(' STEP ===>',step,'; MSE:',round(float(loss_),5), '; RMSE:',round(rmse,5))
    
    # Update the parameters
    step+=1
    optimizer.step()

ValueError: too many dimensions 'str'

In [99]:

class BiasedMatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse=True)
        self.item_biases = torch.nn.Embedding(n_items, 1, sparse=True)

    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (
            (self.user_factors(user) * self.item_factors(item))
            .sum(dim=1, keepdim=True)
        )
        return pred.squeeze()

In [100]:
reg_loss_func = torch.optim.SGD(model.parameters(), lr=1e-6, weight_decay=1e-5)

In [101]:
adagrad_loss = torch.optim.Adagrad(model.parameters(), lr=1e-6)